# Outline
1. Review of inference rules
2. Exercises: Evaluating expressions using inference rules
    1. Well-formed expressions
    2. Typechecking

## Reviewing inference rules

In the past few weeks, you've encountered a variety of formal inference rules of the form

$$\begin{array}{c}
preconditions\\
\hline
postconditions \\
\end{array}\ (\text{Rule Name})$$



In [ ]:
sealed trait Expr

// Atoms
case class Const(v: Double) extends Expr
case object True extends Expr
case object False extends Expr
case class Ident(s: String) extends Expr

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class Div(e1: Expr, e2: Expr) extends Expr
case class Log(e: Expr) extends Expr 
case class Exp(e: Expr) extends Expr
case class Sine(e: Expr) extends Expr
case class Cosine(e: Expr) extends Expr

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr
case class And(e1: Expr, e2: Expr) extends Expr
case class Or(e1: Expr, e2: Expr) extends Expr
case class Not(e: Expr) extends Expr

// Conditional Expressions
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

// Let Binding
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

### Exercise: check if an expression is well-formed

Variables in a Lettuce program are defined before they are used. The inference rules tell us how to implement a function that checks well-formedness. As implementors, we read the rules backwards. For example, $\text{(let-rule)}$ tells us that, in order to check well-formedness of the expression `Let(x, e1, e2)` given the set $S$ of defined variables, we need to
1. confirm that `e1` is well-formed
2. confirm that `e2` is well-formed under defined variables $S \cup \{x\}$

$\fbox{$WellFormed(\texttt{e}, S)$}$

$$ \begin{array}{c}
\\
\hline
WellFormed(\texttt{Const(f)}, S) \\
\end{array} \text{(const-rule)} $$

$$\begin{array}{c}
x \in S \\
\hline
WellFormed(\texttt{Ident(x)}, S) \\
\end{array} \text{(ident-rule)} $$

$$ \begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; WellFormed(\texttt{e2}, S)\;\;\; T \in \{ \texttt{Plus}, \texttt{Minus}, \texttt{Mult}, \texttt{Div}, \texttt{Geq}, \texttt{Eq}, \texttt{And}, \texttt{Or} \} \\
\hline
WellFormed(\texttt{T(e1, e2)}, S) \\
\end{array} \text{(binary-op-rule)} $$

$$ \begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; T \in \{ \texttt{Log}, \texttt{Exp}, \texttt{Sine}, \texttt{Cosine}, \texttt{Not}, \texttt{Eq} \} \\
\hline
WellFormed(\texttt{T(e1)}, S) \\
\end{array} \text{(unary-op-rule)} $$

$$ \begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; WellFormed(\texttt{e2}, S) \;\;\;  WellFormed(\texttt{e3}, S)\\
\hline
WellFormed(\texttt{IfThenElse(e1, e2, e3)}, S) \\
\end{array} \text{(conditional-rule)} $$

$$\begin{array}{c}
\color{blue}{WellFormed(\texttt{e1}, S) \;\;\; WellFormed(\texttt{e2}, S \cup \{ x\} )} \\
\hline
\color{blue}{WellFormed(\texttt{Let(x, e1, e2)}, S)} \\
\end{array} \text{(let-rule)} $$

In [ ]:
def isWellFormed(e: Expr, S: Set[String]): Boolean = {
    
    def isWellFormed_unaryOp(e: Expr) = {
        ???
    }

    def isWellFormed_binOp(e1: Expr, e2: Expr) = {
        ???
    }

    e match {
        case Const(_) | True | False  => true
        case Ident(x) => S contains x
        
        // unary operations
        case Log(e) => isWellFormed_unaryOp(e) 
        case Exp(e) => isWellFormed_unaryOp(e)
        case Sine(e) => isWellFormed_unaryOp(e)
        case Cosine(e) => isWellFormed_unaryOp(e)
        case Not(e) => isWellFormed_unaryOp(e)

        
        // binary operations
        case Plus(e1, e2) => isWellFormed_binOp(e1, e2)
        case Minus(e1, e2) => isWellFormed_binOp(e1, e2)
        case Mult(e1, e2) => isWellFormed_binOp(e1, e2)
        case Div(e1, e2) => isWellFormed_binOp(e1, e2)
        case Geq(e1, e2) => isWellFormed_binOp(e1, e2)
        case Eq(e1, e2) => isWellFormed_binOp(e1, e2)
        case And(e1, e2) => isWellFormed_binOp(e1, e2)
        case Or(e1, e2) => isWellFormed_binOp(e1, e2)
        

        case IfThenElse(e1, e2, e3) => isWellFormed(e1, S) && isWellFormed(e2, S) && isWellFormed(e3, S)
        
        case Let(x, e1, e2) => ???
    }
}


In [ ]:
assert(isWellFormed(Const(1.0), Set()), "Test 1")
assert(isWellFormed(True, Set()), "Test 2")
assert(isWellFormed(Plus(Const(1.0), True), Set()), "Test 3")

assert(isWellFormed(IfThenElse(True, Const(1.0), Const(2.0)), Set()), "Test 4")
assert(!isWellFormed(IfThenElse(True, Const(1.0), Ident("x")), Set()), "Test 5")
assert(isWellFormed(IfThenElse(True, Const(1.0), Ident("x")), Set("x")), "Test 6")
assert(isWellFormed(Let("y", Const(1.0), Ident("y")), Set()), "Test 7")

### Exercise: implement a simple typechecker

We introduce a new function $typeOf(\texttt{e}) = \tau$ that says that the Lettuce expression `e` has type $\tau$. For simplicity, we assume `e` has no `Ident` or `Let` expressions.

We have started defining the inference rules and implementation below. Your task is to extend the inference rules to handle `IfThenElse` expressions. Assume the last two arguments of an `IfThenElse` expression must have the same type.

$\fbox{$typeOf(e) = \tau$}$

$$\begin{array}{c}
\\
\hline
typeOf(\texttt{Const(v)}) = Num \\
\end{array} \text{(const-rule)} $$

$$\begin{array}{c} 
\\
\hline
typeOf(\texttt{True}) = Bool \\
\end{array} \text{(true-rule)} \;\;\;\;
\begin{array}{c} 
\\
\hline
typeOf(\texttt{False}) = Bool \\
\end{array}\text{(false-rule)}
$$

$$\begin{array}{c}
typeOf(\texttt{e1}) = Num\hspace{1.5em} typeOf(\texttt{e2}) = Num\hspace{1.5em} \texttt{T} \in \{ \texttt{Plus, Minus, Mult} \}  \\
\hline
typeOf(\texttt{T(e1, e2)}) = Num \\
\end{array} \text{(arith-binop-ok-rule)}$$

$$\begin{array}{c}
typeOf(\texttt{e1}) \neq Num\hspace{0.5em}\text{or}\hspace{0.5em}typeOf(\texttt{e2}) \neq Num\hspace{1.5em} \texttt{T} \in \{ \texttt{Plus, Minus, Mult} \}  \\
\hline
typeOf(\texttt{T(e1, e2)}) = TypeError \\
\end{array} \text{(arith-binop-nok-rule)}$$

$$\begin{array}{c}
typeOf(\texttt{e1}) = Bool\hspace{1.5em} typeOf(\texttt{e2}) = Bool\hspace{1.5em} \texttt{T} \in \{ \texttt{And, Or} \}  \\
\hline
typeOf(\texttt{T(e1, e2)}) = Bool \\
\end{array} \text{(bool-binop-ok-rule)}$$

$$\begin{array}{c}
typeOf(\texttt{e1}) \neq Bool\hspace{0.5em}\text{or}\hspace{0.5em}typeOf(\texttt{e2}) \neq Bool\hspace{1.5em} \texttt{T} \in \{ \texttt{And, Or} \}  \\
\hline
typeOf(\texttt{T(e1, e2)}) = TypeError \\
\end{array} \text{(bool-binop-nok-rule)}$$



<!-- language: lang-none -->

    preconditions: 

    --------------- (conditional-rule)

    postconditions: 



In [ ]:
def typeOf(e: Expr): String = {
    
    def typeOf_arithBinop(e1 : Expr, e2 : Expr) = {
        if (typeOf(e1) == "Num" && typeOf(e2) == "Num") 
            "Num"
        else 
            "TypeError"
    }
    
    def typeOf_boolBinop(e1 : Expr, e2 : Expr) = {
        if (typeOf(e1) == "Bool" && typeOf(e2) == "Bool") 
            "Bool"
        else 
            "TypeError"
    }
    
    e match {
        // const-rule
        case Const(v) => "Num"
        // true-rule
        case True => "Bool"
        // false-rule
        case False => "Bool"

        // arith-binop-ok-rule and arith-binop-nok-rule
        case Plus(e1,e2) => typeOf_arithBinop(e1, e2)
        case Minus(e1,e2) => typeOf_arithBinop(e1, e2)
        case Mult(e1,e2) => typeOf_arithBinop(e1, e2)

        // bool-binop-ok-rule and bool-binop-nok-rule
        case And(e1, e2) => typeOf_boolBinop(e1, e2)
        case Or(e1, e2) => typeOf_boolBinop(e1, e2)
        
        // conditional-rule
        case IfThenElse(e1, e2, e3) => {
            ???
        }
    }
}

In [ ]:
assert(typeOf(Const(1.0)) == "Num", "Test 1")
assert(typeOf(True) == "Bool", "Test 2")
assert(typeOf(Plus(Const(1.0), Const(2.0))) == "Num", "Test 3")
assert(typeOf(Plus(Const(1.0), True)) == "TypeError", "Test 4")

// Tests your addition
assert(typeOf(IfThenElse(True, Const(1.0), Const(2.0))) == "Num", "Test 5")
assert(typeOf(IfThenElse(Const(0.0), Const(1.0), Const(2.0))) == "TypeError", "Test 6")
assert(typeOf(IfThenElse(False, Const(1.0), True)) == "TypeError", "Test 7")